Fixed length FIFO Queue for Redis

In [200]:
import redis
class redisFIFOQ:
    
    newsQueueName = "news"
    
    def __init__(self, queueSize = 100):
        self.db = redis.Redis()
        self.newsQueueSize = queueSize
    
    def put(self, key, value):
        if not isinstance(value,dict):
            print ("Value needs to be a dictionary " + key)
            return False
        if not isinstance(key,str):
            print ("Key needs to be a string " + key)
            return False
        self.db.hmset(key, value)
        return True
    
    def get(self, key):
        if not isinstance(key,str):
            print ("Key needs to be a string " + key)
            return False
        elif not self.db.exists(key):
            print ("Key doesnt exist " + key)
            return False
        
        val = self.db.hgetall(key)
        return val
    
    def delete(self, key):
        if not self.db.exists(key):
            print ("Key doesnt exist " + key)
            return False
        
        self.db.delete(key)
        return True
    
    def deleteAll(self):
        for key in self.db.keys():
            # delete the key
            self.db.delete(key)
                      
    def checkIfKeyExists(self, key):
        if self.db.exists(key):
            return True
        else:
            return False
        
    ######################Queue starts here###################
    
    def getLengthOfQueue(self):
        return self.db.llen(self.newsQueueName)
    
    #Check if Queue full
    def isQueueFull(self):
        if self.getLengthOfQueue() < self.newsQueueSize:
            return False
        return True
    
    #pushed value from end of queue (Pops oldest news)
    def popQ(self):
        val = self.db.rpop(self.newsQueueName)
        return val 
    
    #pushed value to top of news queue
    def pushQ(self, value): 
        if self.isQueueFull():
            self.popQ()
        self.db.lpush(self.newsQueueName,value)
        return True
    
    #Get values from start to end positions of newws queue
    def getFromQueue(self, start, end):
        val = self.db.lrange(self.newsQueueName, start, end)
        return val
    
    #Get entire news queue content
    def getQ(self):
        return self.getFromQueue(0,self.newsQueueSize-1)

In [207]:
db = redisFIFOQ(5)

In [208]:
db.checkIfKeyExists('Jay')
db.put("Jay", {"surname" : "jaylohokare"})
db.get("Jay")
db.delete("Jay")


True

In [253]:
db.pushQ("50000")

True

In [246]:
db.popQ()

b'5'

In [254]:
db.getQ()

[b'50000']

In [251]:
db.delete('news')

True